In [ ]:
"""
CIFAR-10 Classification (Modified to Dense Layers Only)

Autorzy:
- Kacper Sewruk s23466
- Michał Jastrzemski s26245


Ten notebook wykorzystuje zbiór CIFAR-10 (wbudowany w Keras).
Używa warstw Dense.
Obrazy zostaną spłaszczone i podane do MLP.

Wyniki i wykresy zapisywane są w katalogu `logs/`.
"""

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

os.makedirs('logs', exist_ok=True)

def load_cifar10_data():
    """
    Wczytuje dane CIFAR-10 z Keras.

    Zwraca
    -------
    X_train, X_test : ndarray
        Dane treningowe i testowe o kształcie (N, 32, 32, 3).
    y_train, y_test : ndarray
        Etykiety w postaci numerycznej (0-9).
    """
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    return X_train, X_test, y_train, y_test

def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Przetwarza dane CIFAR-10:
    - Normalizuje piksele do zakresu [0, 1].
    - Konwertuje etykiety do formatu one-hot.
    - Spłaszcza obrazy do wektora 32*32*3 = 3072 cech.

    Parametry
    ---------
    X_train, X_test : ndarray
        Dane treningowe i testowe (obrazki).
    y_train, y_test : ndarray
        Etykiety treningowe i testowe w formie integer.

    Zwraca
    -------
    X_train, X_test, y_train_cat, y_test_cat : ndarray
        Przetworzone dane i etykiety one-hot.
    """
    X_train = X_train.astype('float32')/255.0
    X_test = X_test.astype('float32')/255.0

    # Spłaszczamy obrazki: (32,32,3) -> (3072,)
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)

    y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
    y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

    return X_train, X_test, y_train_cat, y_test_cat

def build_dense_model(input_shape):
    """
    Buduje model MLP do klasyfikacji CIFAR-10:
    - Kilka warstw Dense z aktywacją ReLU
    - Warstwa wyjściowa Softmax (10 klas)

    Parametry
    ---------
    input_shape : tuple
        Kształt wejścia, np. (3072,)

    Zwraca
    -------
    model : tf.keras.Model
        Skompilowany model Dense.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_model(X_train, X_test, y_train_cat, y_test_cat):
    """
    Trenuje i ocenia model MLP na danych CIFAR-10 (spłaszczonych).
    Zapisuje wyniki do:
    - logs/results_cifar10.txt
    - logs/training_history_cifar10.png

    Parametry
    ---------
    X_train, X_test, y_train_cat, y_test_cat : ndarray
        Dane przetworzone i etykiety one-hot.
    """
    model = build_dense_model((X_train.shape[1],))
    history = model.fit(X_train, y_train_cat, validation_split=0.2, epochs=10, batch_size=64, verbose=0)
    loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)

    with open('logs/results_cifar10.txt', 'w') as f:
        f.write(f"Test Accuracy: {acc}\n")

    plt.plot(history.history['accuracy'], label='train acc')
    plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend()
    plt.title('CIFAR-10 Training History (Dense Only)')
    plt.savefig('logs/training_history_cifar10.png')
    plt.show()

def main():
    """
    Funkcja główna:
    1. Wczytuje dane CIFAR-10.
    2. Przetwarza dane (normalizacja, one-hot, spłaszczenie).
    3. Buduje i trenuje model MLP (bez warstw konwolucyjnych).
    4. Zapisuje wyniki.
    """
    X_train, X_test, y_train, y_test = load_cifar10_data()
    X_train, X_test, y_train_cat, y_test_cat = preprocess_data(X_train, X_test, y_train, y_test)
    train_and_evaluate_model(X_train, X_test, y_train_cat, y_test_cat)

main()
